In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import re
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)
timetable_sabato = timetable_domenica = iscrizioni = None
atleti = club = nazionali = nazioni = squadre = categorie = None

def inizializzazione():
    orari_sabato = pd.date_range(start="10:00", end="20:00", freq="5min").strftime('%H:%M')
    orari_domenica = pd.date_range(start="09:30", end="19:00", freq="5min").strftime('%H:%M')
    tatami = ['Tatami ' + str(i) for i in range(1,7)]
    global timetable_sabato
    timetable_sabato = pd.DataFrame(index=orari_sabato, columns=tatami).fillna('')
    global timetable_domenica
    timetable_domenica = pd.DataFrame(index=orari_domenica, columns=tatami).fillna('')

    global iscrizioni
    iscrizioni = pd.read_parquet("iscrizioni.parquet")
    iscrizioni["N"] = iscrizioni["N"].astype(int)
    iscrizioni["Campo9"] = iscrizioni["Campo9"].astype(int)
    iscrizioni["N CLUB"] = iscrizioni["N CLUB"].astype(int)
    iscrizioni["SESSO"] = iscrizioni["SESSO"].str.upper()
    iscrizioni["Kyu/Kup"] = iscrizioni["Kyu/Kup"].astype(int)
    iscrizioni["Dan"] = iscrizioni["Dan"].astype(int)
    iscrizioni["PESO / KG"] = iscrizioni["PESO / KG"].astype(int)
    iscrizioni["ALTEZZA / CM"] = iscrizioni["ALTEZZA / CM"].astype(int)
    iscrizioni["DATA DI NASCITA (AAAA-MM-GG)"] = pd.to_datetime(iscrizioni["DATA DI NASCITA (AAAA-MM-GG)"]).dt.date
    iscrizioni.rename(columns={
        'NAZIONALITÀ': 'NAZIONALITA', 
        'N': 'ID_ATLETA',
        'DATA DI NASCITA (AAAA-MM-GG)': 'DATA_DI_NASCITA',
        'Campo9': 'ID_CATEGORIA',
        'N CLUB': 'ID_CLUB',
        'NATIONAL ID': 'ID_NAZIONALE',
        'PESO / KG': 'PESO_KG',
        'ALTEZZA / CM': 'ALTEZZA_CM',
        'Kyu/Kup': 'KYU_KUP',
        'Dan': 'DAN',
        'TESTA DI SERIE': 'TESTA_DI_SERIE'
    }, inplace=True)

In [2]:
def genera_atleti():
    global iscrizioni
    atleti = iscrizioni.drop_duplicates(subset=['ID_ATLETA']).loc[:, ["ID_ATLETA", "COGNOME", "NOME", "DATA_DI_NASCITA", "SESSO", "KYU_KUP", "DAN", "PESO_KG", "ALTEZZA_CM", "NAZIONALITA", "CLUB", "TESTA_DI_SERIE"]].sort_values(by=['COGNOME', 'NOME'], ascending=True)
    atleti.reset_index(drop=True, inplace=True)
    atleti.set_index("ID_ATLETA", drop=True, inplace=True)
    return atleti

def genera_club():
    global iscrizioni
    club = iscrizioni[['ID_CLUB', 'CLUB']].drop_duplicates().reset_index(drop=True)
    conteggio_atleti = iscrizioni.groupby(['ID_CLUB'])['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
    lista_atleti = iscrizioni.groupby(['ID_CLUB'])['ID_ATLETA'].apply(list).reset_index(name='LISTA_ATLETI')
    club_info = pd.merge(conteggio_atleti, lista_atleti, on='ID_CLUB')
    club = pd.merge(club, club_info, on='ID_CLUB', how='left')
    club.set_index("ID_CLUB", drop=True, inplace=True)
    return club

def genera_nazionali():
    global iscrizioni
    iscrizioni_filtrate = iscrizioni.dropna(subset=['NAZIONALE'])
    nazionali = iscrizioni_filtrate[['NAZIONALE']].drop_duplicates().reset_index(drop=True)
    nazionali.set_index("NAZIONALE", drop=True, inplace=True)
    conteggio_atleti = iscrizioni_filtrate.groupby('NAZIONALE')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
    lista_atleti = iscrizioni_filtrate.groupby('NAZIONALE')['ID_ATLETA'].apply(list).reset_index(name='LISTA_ATLETI')
    nazionali_info = pd.merge(conteggio_atleti, lista_atleti, on='NAZIONALE')
    nazionali = pd.merge(nazionali, nazionali_info, on='NAZIONALE', how='left')
    return nazionali

def genera_nazioni():
    global iscrizioni
    nazioni = iscrizioni[['NAZIONALITA']].drop_duplicates().reset_index(drop=True)
    conteggio_atleti = iscrizioni.groupby('NAZIONALITA')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
    nazioni = pd.merge(nazioni, conteggio_atleti, on='NAZIONALITA', how='left')
    nazioni.set_index("NAZIONALITA", drop=True, inplace=True)
    return nazioni

def genera_squadre():
    global iscrizioni
    iscrizioni_filtrate = iscrizioni.dropna(subset=['SQUADRE'])
    squadre = iscrizioni_filtrate[['SQUADRE']].drop_duplicates().reset_index(drop=True)
    conteggio_atleti = iscrizioni_filtrate.groupby('SQUADRE')['ID_ATLETA'].count().reset_index(name='TOT_ATLETI')
    lista_atleti = iscrizioni_filtrate.groupby('SQUADRE')['ID_ATLETA'].apply(list).reset_index(name='LISTA_ATLETI')
    squadre_info = pd.merge(conteggio_atleti, lista_atleti, on='SQUADRE')
    squadre = pd.merge(squadre, squadre_info, on='SQUADRE', how='left')
    squadre.set_index("SQUADRE", drop=True, inplace=True)
    return squadre

def genera_categorie():
    global iscrizioni
    categorie = iscrizioni.groupby(['CATEGORIA'])['NOME'].count().reset_index(name="ATLETI")
    categorie['TIPO'] = categorie['CATEGORIA'].apply(lambda x: 'KATA' if 'KATA' in x else ('KUMITE' if 'KUMITE' in x else 'ALTRO'))
    categorie['TEAM'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'TEAM' in x else '')
    categorie['CINTURE'] = categorie['CATEGORIA'].apply(
        lambda x: 'NERE' if 'BLACK' in x else (
            'COLORATE' if re.search('YELLOW|ORANGE|GREEN|BLUE|BROWN', x) else 'TUTTE'
        )
    )
    categorie['YELLOW'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'YELLOW' in x else '')
    categorie['ORANGE'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'ORANGE' in x else '')
    categorie['GREEN'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'GREEN' in x else '')
    categorie['BLUE'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'BLUE' in x else '')
    categorie['BROWN'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'BROWN' in x else '')
    categorie['BLACK'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'BLACK' in x else '')
    
    categorie['KIDS'] = categorie['CATEGORIA'].apply(lambda x: 'x' if 'KIDS' in x else '')
    def check_over(x):
        cleaned_x = re.sub(r'\bOVER A\b', '', x)
        return 'x' if re.search(r'\bOVER\b', cleaned_x) else ''
    
    def check_over_a(x):
        return 'x' if re.search(r'\bOVER A\b', x) else ''
    categorie['OVER'] = categorie['CATEGORIA'].apply(check_over)
    categorie['OVER A'] = categorie['CATEGORIA'].apply(check_over_a)
    
    iscrizioni_filtrate = iscrizioni.dropna(subset=['SQUADRE'])
    grouped_squadre = iscrizioni_filtrate.groupby('CATEGORIA')['SQUADRE'].nunique().reset_index()
    categorie = pd.merge(categorie, grouped_squadre, left_on='CATEGORIA', right_on='CATEGORIA', how='left')
    categorie['SQUADRE'] = categorie['SQUADRE'].fillna(0).astype(int)
    
    over32 = [
        (1, 33),
        (17, 49),
        (9, 41),
        (25, 57),
        (5, 37),
        (21, 53),
        (13, 45),
        (29, 61),
        (3, 35),
        (19, 51),
        (11, 43),
        (27, 59),
        (7, 39),
        (23, 55),
        (15, 47),
        (31, 63),
        (2, 34),
        (18, 50),
        (10, 42),
        (26, 58),
        (6, 38),
        (22, 54),
        (14, 46),
        (30, 62),
        (4, 36),
        (20, 52),
        (12, 44),
        (28, 60),
        (8, 40),
        (24, 56),
        (16, 48),
        (32, 64)
    ]
    
    sedicesimi = [
        (1, 17),
        (9, 25),
        (5, 21), 
        (13, 29),
        (3, 19),
        (11, 27),
        (7, 23),
        (15, 31),
        (2,  18),
        (10, 26),
        (6, 22),
        (14, 30),
        (4, 20),
        (12, 28),
        (8, 24),
        (16, 32)
    ]
    
    ottavi = [
        (1, 9),
        (5, 13),
        (3, 11),
        (7, 15),
        (2, 10),
        (6, 14),
        (4, 12),
        (8, 16)
    ]
    
    quarti = [
        (1, 5),
        (3, 7),
        (2, 6),
        (4, 8)
    ]
    
    semifinale = [
        (1, 3), 
        (2, 4)
    ]
    
    finale = [
        (1, 2)
    ]
    
    # Variabili di tempo in minuti
    tempo_kata_kids_over = 2 #da moltiplicare per 2 prove
    tempo_kata_nere_bandierine = 2
    tempo_kata_nere_punteggio = 2.5
    tempo_kata_colorate = 2.5
    tempo_kata_team = 2.5
    tempo_kumite = 3.5  
    
    # Funzione per calcolare il tempo totale per ciascuna categoria
    def calcola_tempo(row):
        # KATA
        if row["TIPO"] == "KATA":
            # SQUADRE
            if row["TEAM"] == "x":
                return 0
            # INDIVIDUALI 
            else:
                # KIDS, OVER e OVER A a punteggio
                if row["KIDS"] == "x" or row["OVER"] == "x" or row["OVER A"] == "x":
                    return math.ceil(row["ATLETI"] * tempo_kata_kids_over)
                # ALTRE ETA' CINTURE COLORATE
                elif row["CINTURE"] == "COLORATE":
                    return math.ceil(row["ATLETI"] * tempo_kata_colorate)
                # ALTRE ETA' CINTURE NERE
                elif row["CINTURE"] == "NERE":
                    n_atleti = row["ATLETI"]            
                    tempo = 0
                    # Più di 32 atleti, vedi prima colonna del tabellone gare - BANDIERINE
                    if n_atleti > 32:
                        incontri_over32 = [(a, b) for a, b in over32 if a <= n_atleti and b <= n_atleti]
                        n_incontri = len(incontri_over32)
                        tempo += n_incontri * tempo_kata_nere_bandierine
                        n_atleti = 32
                    # Fra 32 e 16 atleti, vedi colonna "sedicesimi" - BANDIERINE
                    if n_atleti > 16:
                        incontri_sedicesimi = [(a, b) for a, b in sedicesimi if a <= n_atleti and b <= n_atleti]
                        n_incontri = len(incontri_sedicesimi)
                        tempo += n_incontri * tempo_kata_nere_bandierine
                        n_atleti = 16
                    # Fra 16 e 8 atleti, vedi colonna "ottavi" - BANDIERINE
                    if n_atleti > 8:
                        incontri_ottavi = [(a, b) for a, b in ottavi if a <= n_atleti and b <= n_atleti]
                        n_incontri = len(incontri_ottavi)
                        tempo += n_incontri * tempo_kata_nere_bandierine
                        n_atleti = 8
                    # Fra 8 e 4 atleti, vedi colonna "quarti" - PUNTEGGIO
                    if n_atleti > 4:
                        n_incontri = n_atleti # "incontri singoli", ogni atleta gareggia da solo
                        tempo += n_incontri * tempo_kata_nere_punteggio
                        n_atleti = 4
                    # SEMIFINALE - PUNTEGGIO
                    n_incontri = n_atleti # "incontri singoli", ogni atleta gareggia da solo
                    tempo += n_incontri * tempo_kata_nere_punteggio
                    n_atleti = 2
                    # FINALE - PUNTEGGIO
                    n_incontri = n_atleti # "incontri singoli", ogni atleta gareggia da solo
                    tempo += n_incontri * tempo_kata_nere_punteggio                  
    
                    return math.ceil(tempo)
                
                # ERRORE
                else:
                    return -1
        # KUMITE
        else:
            # SQUADRE
            if row["TEAM"] == "x":
                return 0
            # INDIVIDUALI 
            else:
                n_atleti = row["ATLETI"]            
                tempo = 0
                # Più di 32 atleti, vedi prima colonna del tabellone gare 
                if n_atleti > 32:
                    incontri_over32 = [(a, b) for a, b in over32 if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_over32)
                    tempo += n_incontri * tempo_kumite
                    n_atleti = 32
                # Fra 32 e 16 atleti, vedi colonna "sedicesimi"
                if n_atleti > 16:
                    incontri_sedicesimi = [(a, b) for a, b in sedicesimi if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_sedicesimi)
                    tempo += n_incontri * tempo_kumite
                    n_atleti = 16
                # Fra 16 e 8 atleti, vedi colonna "ottavi"
                if n_atleti > 8:
                    incontri_ottavi = [(a, b) for a, b in ottavi if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_ottavi)
                    tempo += n_incontri * tempo_kumite
                    n_atleti = 8
                # Fra 8 e 4 atleti, vedi colonna "quarti" 
                if n_atleti > 4:
                    incontri_quarti = [(a, b) for a, b in quarti if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_quarti)
                    tempo += n_incontri * tempo_kumite
                    n_atleti = 4
                # SEMIFINALE 
                if n_atleti > 2:
                    incontri_semifinale = [(a, b) for a, b in semifinale if a <= n_atleti and b <= n_atleti]
                    n_incontri = len(incontri_semifinale)
                    tempo += n_incontri * tempo_kumite
                    n_atleti = 2
                # FINALE 
                tempo += tempo_kumite                  
    
                return math.ceil(tempo)
                
    categorie['TEMPO (MINUTI)'] = categorie.apply(lambda row: calcola_tempo(row), axis=1)
    categorie.set_index("CATEGORIA", drop=True, inplace=True)
    return categorie 

def aggiorna_tutto():
    global atleti 
    atleti = genera_atleti()
    global club 
    club = genera_club()
    global nazionali
    nazionali = genera_nazionali()
    global nazioni
    nazioni = genera_nazioni()
    global squadre
    squadre = genera_squadre()
    global categorie
    categorie = genera_categorie()


In [3]:
def aggiungi_categoria_a_timetable(timetable, categoria, tempo_categoria, orario_inizio=None, tatami=None):
    """
    Aggiunge una categoria al timetable specificato.

    Parametri:
    - timetable: DataFrame del timetable (sabato o domenica)
    - categoria: Nome della categoria da aggiungere
    - tempo_categoria: Tempo in minuti necessario per la categoria
    - orario_inizio: Orario di inizio per la categoria (formato 'HH:MM'). Default: primo disponibile
    - tatami: Tatami specifico in cui inserire la categoria. Default: primo disponibile

    Ritorna:
    - Nuovo DataFrame del timetable aggiornato
    """

    # Arrotonda il tempo della categoria al multiplo di 5 minuti successivo
    num_slots = math.ceil(tempo_categoria / 5)
    
    # Se un tatami specifico è fornito
    if tatami:
        if tatami not in timetable.columns:
            raise ValueError(f"Il tatami {tatami} non è nel timetable.")
        
        # Se anche l'orario di inizio è fornito
        if orario_inizio:
            if orario_inizio not in timetable.index:
                raise ValueError(f"L'orario di inizio {orario_inizio} non è nel timetable.")
            
            orario_corrente = pd.to_datetime(orario_inizio)
            for j in range(num_slots):
                slot_corrente = orario_corrente.strftime('%H:%M')
                if slot_corrente in timetable.index:
                    timetable.loc[slot_corrente, tatami] = categoria
                orario_corrente += pd.Timedelta(minutes=5)
                
            return timetable
            
        # Se solo il tatami è fornito, trova il primo orario disponibile
        else:
            for i, (index, row) in enumerate(timetable.iterrows()):
                if row[tatami] == '':
                    orario_corrente = pd.to_datetime(index)
                    for j in range(num_slots):
                        slot_corrente = orario_corrente.strftime('%H:%M')
                        if slot_corrente in timetable.index:
                            timetable.loc[slot_corrente, tatami] = categoria
                        orario_corrente += pd.Timedelta(minutes=5)
                    
                    return timetable

    # Se nessun tatami è fornito, comportamento predefinito
    else:
        for i, (index, row) in enumerate(timetable.loc[orario_inizio:].iterrows() if orario_inizio else timetable.iterrows()):
            slots_disponibili = row.isin(['']).sum()
            if slots_disponibili >= 1:
                tatami_disponibili = [tatami for tatami, disponibile in row.items() if disponibile == '']
                tatami_selezionato = tatami_disponibili[0]
                
                orario_corrente = pd.to_datetime(index)
                for j in range(num_slots):
                    slot_corrente = orario_corrente.strftime('%H:%M')
                    if slot_corrente in timetable.index:
                        timetable.loc[slot_corrente, tatami_selezionato] = categoria
                    orario_corrente += pd.Timedelta(minutes=5)
                
                return timetable



In [4]:
def dividi_in_pool(nome_categoria):
    global iscrizioni

    # Filtra gli atleti della categoria specificata
    atleti_categoria = iscrizioni[iscrizioni['CATEGORIA'] == nome_categoria]
    
    if atleti_categoria.empty:
        raise ValueError(f"La categoria {nome_categoria} non esiste.")
    
    # Inizializza le liste per i pool
    pool_1 = []
    pool_2 = []
    
    # Divide gli atleti di ogni club tra i due pool, cercando di mantenere un numero uguale
    for _, gruppo in atleti_categoria.groupby('ID_CLUB'):
        gruppo = gruppo.sort_values(by='ID_CLUB')
        for _, atleta in gruppo.iterrows():
            if len(pool_1) <= len(pool_2):
                pool_1.append(atleta)
            else:
                pool_2.append(atleta)

    # Converti le liste in DataFrame
    pool_1_df = pd.DataFrame(pool_1).reset_index(drop=True)
    pool_2_df = pd.DataFrame(pool_2).reset_index(drop=True)
    
    # Aggiorna i nomi delle categorie nei DataFrame dei pool
    pool_1_df['CATEGORIA'] = nome_categoria + '_POOL_1'
    pool_2_df['CATEGORIA'] = nome_categoria + '_POOL_2'
    
    # Rimuovi la vecchia categoria e aggiungi i nuovi pool al DataFrame delle iscrizioni
    iscrizioni = iscrizioni[iscrizioni['CATEGORIA'] != nome_categoria]
    iscrizioni = pd.concat([iscrizioni, pool_1_df, pool_2_df])

    aggiorna_tutto()

def unisci_pool(nome_categoria):
    global iscrizioni

    # Trova i nomi delle categorie dei pool
    nome_pool_1 = nome_categoria + '_POOL_1'
    nome_pool_2 = nome_categoria + '_POOL_2'

    # Estrai i DataFrame dei due pool
    pool_1 = iscrizioni[iscrizioni['CATEGORIA'] == nome_pool_1]
    pool_2 = iscrizioni[iscrizioni['CATEGORIA'] == nome_pool_2]

    # Controlla se i pool esistono
    if pool_1.empty and pool_2.empty:
        raise ValueError(f"I pool per la categoria {nome_categoria} non sono stati trovati.")

    # Unisci i due pool in un unico DataFrame
    pool_unificato = pd.concat([pool_1, pool_2])

    # Cambia il nome della categoria nel DataFrame unificato
    pool_unificato['CATEGORIA'] = nome_categoria

    # Rimuovi i vecchi pool e inserisci la categoria unificata
    iscrizioni = iscrizioni[(iscrizioni['CATEGORIA'] != nome_pool_1) & (iscrizioni['CATEGORIA'] != nome_pool_2)]
    iscrizioni = pd.concat([iscrizioni, pool_unificato])

    aggiorna_tutto()



In [5]:
inizializzazione()
aggiorna_tutto()